In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

In [2]:
# calculate_metrics function will take the actual and predicted values as input
# and return overall accuracy, precision and recall as output
def calculate_metrics(actual, predicted):

    y_actu = pd.DataFrame(actual, columns=["Actual"])
    y_pred = pd.DataFrame(predicted, columns=["Predicted"])
    
    confusion_matrix = pd.crosstab(y_actu["Actual"], y_pred["Predicted"])

    FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)
    FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
    TP = np.diag(confusion_matrix)
    TN = confusion_matrix.values.sum() - (FP + FN + TP)
    
    del(confusion_matrix)

    # Recall, the proportion of true positive predictions
    # out of all actual positive cases.
    recall = TP / (TP + FN) * 100

    # Precision, the proportion of true positive predictions
    # out of all positive predictions made by the model.
    precision = TP / (TP + FP) * 100

    # Accuracy, the proportion of correct predictions made by the model.
    accuracy = (TP + TN) / (TP + FP + FN + TN) * 100

    # These values are calculated for each class, and then averaged to get the overall score.
    
    return (recall.mean(), precision.mean(), accuracy.mean())

I used matrix multiplication: euclidean distance matrix = sqrt(p1 + p2 + p3)


p1 = sum of squares of rows of train matrix

p2 = sum of squares of rows of test matrix

p3 = -2 * dot product of rows of train matrix and transpose of test matrix

If I have two test vectors and 8 train vectors, than I'll recieve a matrix that consists of 8 rows and 2 columns, each column has the distances of a test vector to every train vector. For example:

                     test_vector_1 test_vector_2
                 
    train_vector_1   [[9.69535971  14.31782106]

    train_vector_2   [10.44030651 14.76482306]

    train_vector_3   [10.39230485 14.45683229]

        etc.         [11.87434209 14.69693846]

                     [8.83176087  9.21954446]

                     [9.16515139  12.68857754]

                     [11.87434209 16.30950643]

                     [8.60232527  14.59451952]]


In [3]:
def most_frequent(a, axis=0):
    scores = np.unique(np.ravel(a))  # unique values
    testshape = list(a.shape)
    testshape[axis] = 1
    oldmostfreq = np.zeros(testshape)
    oldcounts = np.zeros(testshape)

    for score in scores:
        template = a == score
        counts = np.expand_dims(np.sum(template, axis), axis)
        mostfrequent = np.where(counts > oldcounts, score, oldmostfreq)
        oldcounts = np.maximum(counts, oldcounts)
        oldmostfreq = mostfrequent

    return mostfrequent

In [4]:
def predict(train_data, test_data, k):
    # keep the Personality type column in a separate variable
    train_personalities = train_data[:, -1]

    # drop the Personality type column from the train data
    train_data = train_data[:, :-1]

    # calculate the first part of the distance formula
    first_term = np.sum(train_data**2, axis=1)[:, np.newaxis]

    # calculate the second part of the distance formula
    second_term = np.sum(test_data**2, axis=1)

    # calculate the third part of the distance formula
    third_term = -2 * np.dot(train_data, test_data.T)
    
    del train_data #trying to optimize memory
    
    # calculate the Euclidean distance matrix
    distance_matrix = np.sqrt(first_term + second_term + third_term)

    # get the indices of the k nearest neighbors for each column in the test data
    k_nearest_neighbors = np.argsort(distance_matrix, axis=0)[:k]
    
    del distance_matrix #trying to optimize memory
    
    # get the personalities (labels) of the k nearest neighbors
    nearest_personalities = train_personalities[k_nearest_neighbors]
    
    #find the most frequent element in each column
    predicted = most_frequent(nearest_personalities)[0]
    
    return predicted

I use 5 fold cross validation function five_fold_cross() which splits data to 5 folds (12.000 row each fold). 
First, I split my data array to 5 pieces with np.array_split(). Then, I use a for loop to iterate through folds. In every iteration: 

I choose one fold as my test_data, and use np.concatenate() to combine other 4 folds to use as train_data. 

In [5]:
def five_fold_cross_val_score(data, k, predict_func):
    fold = 5

    # split the data into folds
    folds_list = np.array_split(data, fold)  # 5 folds of 12.000 rows

    total_accuracy = 0
    total_precision = 0
    total_recall = 0

    # iterate over the folds
    for i in range(fold):
        # concatenate the folds except the current one to get the training data
        train = np.concatenate(folds_list[:i] + folds_list[i + 1 :])

        # get the test data
        test = folds_list[i]

        # actual Personality type
        actual = test[:, -1]

        # drop the Personality type column
        test = test[:, :-1]
        
        predicted = predict_func(train, test, k)
    

        # current accuracy, precision, recall
        (cur_rec, cur_pre, cur_acc) = calculate_metrics(actual, predicted)

        total_accuracy += cur_acc
        total_precision += cur_pre
        total_recall += cur_rec
        
    return (total_accuracy / fold, total_precision / fold, total_recall / fold)

In [6]:
# Dictionary for personality types
personality_dict = {
    "ESTJ": 0,
    "ENTJ": 1,
    "ESFJ": 2,
    "ENFJ": 3,
    "ISTJ": 4,
    "ISFJ": 5,
    "INTJ": 6,
    "INFJ": 7,
    "ESTP": 8,
    "ESFP": 9,
    "ENTP": 10,
    "ENFP": 11,
    "ISTP": 12,
    "ISFP": 13,
    "INTP": 14,
    "INFP": 15,
}


# nrows is max read rows
df = pd.read_csv("16P.csv", encoding="cp1252")

# drop response id
df = df.drop("Response Id", axis="columns")

# encode "personality" column of df to numbers in typeDict
df["Personality"] = df["Personality"].map(personality_dict)

In [7]:
#convert dataframe to numpy array
df_arr = df.values

In [8]:
# k values
k_values = [1, 3, 5, 7, 9]

# accuracy values corresponding to each k value
accuracy = []

# accuracy values corresponding to each k value
precision = []

# accuracy values corresponding to each k value
recall = []

#runtime corresponding to each k value
times = []


In [ ]:
for k in k_values:
    print("For k =", k)
    start = time.time()
    (acc, prec, rec) = five_fold_cross_val_score(df_arr, k,predict)
    end = time.time()
    print("Recall:", rec)
    print("Precision:", prec)
    print("Accuracy:", acc)
    print("Elapsed time:", time.strftime("%H:%M:%S", time.gmtime(end - start)), "\n")
    
    accuracy.append(acc)
    precision.append(prec)
    recall.append(rec)
    times.append(end-start)


For k = 1


In [ ]:
plt.plot(k_values, accuracy)

plt.title("Accuracy - k Graph")
plt.xlabel("k")
plt.ylabel("Accuracy")

plt.show()


plt.plot(k_values, precision)

plt.title("Precision - k Graph")
plt.xlabel("k")
plt.ylabel("Precision")

plt.show()


plt.plot(k_values, recall)

plt.title("Recall - k Graph")
plt.xlabel("k")
plt.ylabel("Recall")

plt.show()


plt.plot(k_values, times)

plt.title("Time - k Graph")
plt.xlabel("k")
plt.ylabel("Time")

plt.show()

In [ ]:
def weighted_predict(train_data, test_data, k):
    pred_list = [] 
    
    # keep the Personality type column in a separate variable
    train_personalities = train_data[:, -1]

    # drop the Personality type column from the train data
    train_data = train_data[:, :-1]

    # calculate the first part of the distance formula
    first_term = np.sum(train_data**2, axis=1)[:, np.newaxis]

    # calculate the second part of the distance formula
    second_term = np.sum(test_data**2, axis=1)

    # calculate the third part of the distance formula
    third_term = -2 * np.dot(train_data, test_data.T)
    
    del train_data #trying to optimize memory

    # calculate the Euclidean distance matrix
    distance_matrix = np.sqrt(first_term + second_term + third_term)

    # get the indices of the k nearest neighbors for each column in the test data
    k_nearest_neighbors = np.argsort(distance_matrix, axis=0)[:k]
    
    del distance_matrix #trying to optimize memory
    
    # get the personalities (labels) of the k nearest neighbors
    nearest_personalities = train_personalities[k_nearest_neighbors]
    
    #get weights
    weights = (1/k_nearest_neighbors)
    
    for col in range(test_data.shape[1]):
        #key is the personality type, value is the total weight
        weight_dict = {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0}

        #key is the personality type, value is the number it's occured
        encounter_dict = {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0}

        for i in range(len(nearest_personalities[col])):
            weight_dict[nearest_personalities[col][i]] += weights[col][i]
            encounter_dict[nearest_personalities[col][i]] += 1

        for j in range(len(nearest_personalities[col])):
            weight_dict[nearest_personalities[col][i]] = weight_dict[nearest_personalities[col][i]] / encounter_dict[nearest_personalities[col][i]]

        predicted = 0
        max_value = -1
        max_index = -1
        values = list(weight_dict.values())

        for k in range(len(values)):
            if values[k] >=  max_value:
                max_value = values[k]
                max_index = k

        predicted = list(weight_dict.keys())[max_index]
        pred_list.append(predicted)
    
    return pred_list

In [ ]:
# accuracy values corresponding to each k value
accuracy_wkk = []

# accuracy values corresponding to each k value
precision_wkk = []

# accuracy values corresponding to each k value
recall_wkk = []

#runtime corresponding to each k value
times_wkk = []

In [ ]:
for k in k_values:
    print("For k =", k)
    start = time.time()
    (acc, prec, rec) = five_fold_cross_val_score(df_arr, k,weighted_predict)
    end = time.time()
    print("Recall:", rec)
    print("Precision:", prec)
    print("Accuracy:", acc)
    print("Elapsed time:", time.strftime("%H:%M:%S", time.gmtime(end - start)), "\n")
    
    accuracy_wkk.append(acc)
    precision_wkk.append(prec)
    recall_wkk.append(rec)
    times_wkk.append(end-start)

In [ ]:
plt.plot(k_values, accuracy_wkk)

plt.title("Weighted Accuracy - k Graph")
plt.xlabel("k")
plt.ylabel("Accuracy")

plt.show()


plt.plot(k_values, precision_wkk)

plt.title("Weighted Precision - k Graph")
plt.xlabel("k")
plt.ylabel("Precision")

plt.show()


plt.plot(k_values, recall_wkk)

plt.title("Weighted Recall - k Graph")
plt.xlabel("k")
plt.ylabel("Recall")

plt.show()


plt.plot(k_values, times_wkk)

plt.title("Weighted Time - k Graph")
plt.xlabel("k")
plt.ylabel("Time")

plt.show()

feature_scale function will apply a min-max normalization on the data. 
Every row has 60 independent variables, a.k.a. 60 feature columns, each feature column will be scaled between 
0 and 1 using min-max normalization with the formula: 

scaled_row = (row - min(row)) / (max(row) - min(row))

where min(row) is the minimum value of the row and max(row) is the maximum value of the row. 
In our case, min(row) is always -3 and max(row) is always 3

In [ ]:
def feature_scale(data):
    possible_min = -3
    possible_max = 3

    denominator = (possible_max - possible_min)  # didn't want to calculate this every time since it's constant


    return (data - possible_min) / (denominator)

In [ ]:
#keep the personality colunmn so that it won't get normalized
temp_personality_column = df_arr[:, -1]

#drop the personality column
df_arr = df_arr[:, :-1]

#scale the array
df_arr = feature_scale(df_arr)

#add the original personality column again
df_arr = np.column_stack((df_arr, temp_personality_column))

In [ ]:
# accuracy values corresponding to each k value
accuracy_scaled = []

# accuracy values corresponding to each k value
precision_scaled = []

# accuracy values corresponding to each k value
recall_scaled = []

#runtime corresponding to each k value
times_scaled = []

In [ ]:
print("After data is SCALED\n---------------\n")
for k in k_values:
    print("For k =", k)
    start = time.time()
    (acc, prec, rec) = five_fold_cross_val_score(df_arr, k)
    end = time.time()
    print("Recall:", rec)
    print("Precision:", prec)
    print("Accuracy:", acc)
    print("Elapsed time:", time.strftime("%H:%M:%S", time.gmtime(end - start)), "\n")
    
    accuracy_scaled.append(acc)
    precision_scaled.append(prec)
    recall_scaled.append(rec)
    times_scaled.append(end-start)

In [ ]:
plt.plot(k_values, accuracy, color='r', label='No scale')
plt.plot(k_values, accuracy_scaled, color='g', label='Scaled')
  
plt.xlabel("K")
plt.ylabel("Accuracy")
plt.title("Change of Accuracy With Normalization")
  
plt.legend()

plt.show()



plt.plot(k_values, precision, color='r', label='No scale')
plt.plot(k_values, precision_scaled, color='g', label='Scaled')

plt.xlabel("K")
plt.ylabel("Precision")
plt.title("Change of Precision With Normalization")

plt.legend()
  
plt.show()


plt.plot(k_values, recall, color='r', label='No scale')
plt.plot(k_values, recall_scaled, color='g', label='Scaled')
  
plt.xlabel("K")
plt.ylabel("Recall")
plt.title("Change of Recall With Normalization")
  
plt.legend()
  
plt.show()



plt.plot(k_values, times, color='r', label='No scale')
plt.plot(k_values, times_scaled, color='g', label='Scaled')
  
plt.xlabel("K")
plt.ylabel("Runtime (minutes)")
plt.title("Change of Runtime With Normalization")
  
plt.legend()
  
plt.show()

Error Analysis

I observed some miscorrect guesses while testing. I realised some possible reasons:

    1. Since the algorithm is based on finding nearest neighbours and choosing the most frequently seen among them (plurality voting), if we don't have a frequent neighbour (for example if we're experimenting with k = 3 and every one of the k - nearest distances are unique), then my model tend to make mistakes. Because I use argmax to find the maximum repeated distance, and it'll return the first occured if their frequencies is same.  ( np.argmax([1,1,1]) --> 0 )

    2. Especially using k = 1 drastically decreases my model's correctness. Which that makes sence since I just sort the distances and get the first lowest distance, don't check if there're any other distances, don't check the frequencies etc. 

    3. I tried to check if the misclassified samples all have similar characteristics, couldn't find a strong correlation among them that's strong enough to convince me. I checked my confusion matrix (16 x 16, prints the actual and predicted values with respect to each personality), errors seemed homogenously distributed. 

    4. Another hypothesis of mine was if there's a case of "class imbalance", that is if one class is overrepresented, making it hard for my model to learn the other classes. That doesn't seem like the case neither. Every personality type seems almost equally distrubited to data.


Performance Analysis

All of my recall, precision and accuracy values seems to be increasing with k value between 1 - 5. So k = 3 is better than k = 1, k = 5 is better than k = 3. Between 5 - 7, increase continues but acceleration decreases. 

Also, all of the values are better in general when data is scaled.

When it comes to time, it has a sharp decrease between K = 1 and K = 3, which I don't know, it seems counter-intuitive to me. It may be related to my system. Between scaled and non-scaled data, scaled performs better in terms of running time. 
